## Import libraries

In [1]:
import pandas as pd
import numpy as np
import numpyro
import numpyro.distributions as dist
from numpyro.distributions import Distribution, constraints
from numpyro.distributions.util import validate_sample
from numpyro.infer import MCMC, NUTS, Predictive, init_to_median
import jax
from jax import random
from jax.scipy.stats import gaussian_kde
import jax.numpy as jnp
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az
import os
import pickle
import yaml

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Create random seed for JAX
rng_key = random.PRNGKey(0)

## Define Models

In [4]:
DISTRIBUTIONS = {
    "normal": dist.Normal,
    "half_normal": dist.HalfNormal,
    "student_t": dist.StudentT,
    "laplace": dist.Laplace,
    "uniform": dist.Uniform,
    "gamma": dist.Gamma
}

In [5]:
def pooled(X, y, ind, features_names, from_posterior=None, **init_params_kwargs):
    prior_dist = init_params_kwargs.get("prior_dist", "normal")
    prior_params = init_params_kwargs.get("prior_params", {"loc": 0, "scale": 1})
    shape_dist = init_params_kwargs.get("shape_dist", "uniform")
    shape_params = init_params_kwargs.get("shape_params", {"low": 1, "high": 100})
    target_dist = init_params_kwargs.get("target_dist", "gamma")

    if from_posterior is None:
        avg_salary = numpyro.sample("avg_salary", DISTRIBUTIONS[prior_dist](**prior_params))
        priors = []
        for i, feature in enumerate(features_names):
            priors.append(numpyro.sample(f"beta_{feature}", DISTRIBUTIONS[prior_dist](**prior_params)))
    else:
        avg_salary = numpyro.sample("avg_salary", DISTRIBUTIONS[prior_dist](from_posterior["avg_salary"].mean(), from_posterior["avg_salary"].std()))
        priors = []
        for i, feature in enumerate(features_names):
            priors.append(numpyro.sample(f"beta_{feature}", DISTRIBUTIONS[prior_dist](from_posterior[f"beta_{feature}"].mean(), from_posterior[f"beta_{feature}"].std())))
    shape = numpyro.sample("shape", DISTRIBUTIONS[shape_dist](**shape_params))

    # Expected value
    mu = avg_salary
    for i, prior in enumerate(priors):
        mu += prior * X[:,i]
    mu = jnp.exp(mu)
    rate = shape / mu

    # Likelihood
    with numpyro.plate("data", X.shape[0]):
        numpyro.sample("salary_hat", DISTRIBUTIONS[target_dist](concentration=shape, rate=rate), obs=y)

def no_pooled(X, y, ind, features_names, from_posterior=None, **init_params_kwargs):
    # Initial parameters
    prior_dist = init_params_kwargs.get("prior_dist", "normal")
    prior_params = init_params_kwargs.get("prior_params", {"loc": 0, "scale": 1})
    shape_dist = init_params_kwargs.get("shape_dist", "uniform")
    shape_params = init_params_kwargs.get("shape_params", {"low": 1, "high": 100})
    target_dist = init_params_kwargs.get("target_dist", "gamma")

    # Priors
    if from_posterior is None:
        with numpyro.plate("industry", 16):
            avg_salary = numpyro.sample("avg_salary", DISTRIBUTIONS[prior_dist](**prior_params))
            priors = []
            for i, feature in enumerate(features_names):
                priors.append(numpyro.sample(f"beta_{feature}", DISTRIBUTIONS[prior_dist](**prior_params)))
    else:
        with numpyro.plate("industry", 16):
            avg_salary = numpyro.sample("avg_salary", 
                                        DISTRIBUTIONS[prior_dist](from_posterior["avg_salary"].mean(axis=0), from_posterior["avg_salary"].std(axis=0)))
            priors = []
            for i, feature in enumerate(features_names):
                priors.append(numpyro.sample(f"beta_{feature}", 
                                             DISTRIBUTIONS[prior_dist](from_posterior[f"beta_{feature}"].mean(axis=0), from_posterior[f"beta_{feature}"].std(axis=0))))
    shape = numpyro.sample("shape", DISTRIBUTIONS[shape_dist](**shape_params))

    # Expected value
    mu = avg_salary[ind]
    for i, prior in enumerate(priors):
        mu += prior[ind] * X[:,i]
    mu = jnp.exp(mu)
    rate = shape / mu

    # Likelihood
    with numpyro.plate("data", X.shape[0]):
        numpyro.sample("salary_hat", DISTRIBUTIONS[target_dist](concentration=shape, rate=rate), obs=y)

def hierarchical(X, y, ind, features_names, from_posterior=None, **init_params_kwargs):
    # Initial parameters
    mu_dist = init_params_kwargs.get("mu_dist", "normal")
    mu_params = init_params_kwargs.get("mu_params", {"loc": 0, "scale": 3})
    sigma_dist = init_params_kwargs.get("sigma_dist", "half_normal")
    sigma_params = init_params_kwargs.get("sigma_params", {"scale": 3})
    shape_dist = init_params_kwargs.get("shape_dist", "uniform")
    shape_params = init_params_kwargs.get("shape_params", {"low": 1, "high": 100})
    target_dist = init_params_kwargs.get("target_dist", "gamma")

    # Hyperpriors
    mus = []
    sigmas = []
    if from_posterior is None:
        mu_avg_salary = numpyro.sample("mu_avg_salary", DISTRIBUTIONS[mu_dist](**mu_params))
        sigma_avg_salary = numpyro.sample("sigma_avg_salary", DISTRIBUTIONS[sigma_dist](**sigma_params))
        
        for feature in features_names:
            mus.append(numpyro.sample(f"mu_{feature}", DISTRIBUTIONS[mu_dist](**mu_params)))
            sigmas.append(numpyro.sample(f"sigma_{feature}", DISTRIBUTIONS[sigma_dist](**sigma_params)))
    else:
        mu_avg_salary = numpyro.sample("mu_avg_salary", 
                                       DISTRIBUTIONS[mu_dist](from_posterior["mu_avg_salary"].mean(axis=0), from_posterior["mu_avg_salary"].std(axis=0)))
        sigma_avg_salary = numpyro.sample("sigma_avg_salary", 
                                          DISTRIBUTIONS[sigma_dist](from_posterior["sigma_avg_salary"].mean(axis=0)))
        
        for feature in features_names:
            mus.append(numpyro.sample(f"mu_{feature}", 
                                      DISTRIBUTIONS[mu_dist](from_posterior[f"mu_{feature}"].mean(axis=0), from_posterior[f"mu_{feature}"].std(axis=0))))
            sigmas.append(numpyro.sample(f"sigma_{feature}", 
                                         DISTRIBUTIONS[sigma_dist](from_posterior[f"sigma_{feature}"].mean(axis=0))))

    with numpyro.plate(f"industry", 16):
        offset_avg_salary = numpyro.sample("offset_avg_salary", DISTRIBUTIONS["normal"](loc=0, scale=1))
        avg_salary = numpyro.deterministic("avg_salary", mu_avg_salary + offset_avg_salary * sigma_avg_salary)
        priors = []
        for i, feature in enumerate(features_names):
            offset = numpyro.sample(f"offset_{feature}", DISTRIBUTIONS["normal"](loc=0, scale=1))
            priors.append(numpyro.deterministic(f"beta_{feature}", mus[i] + offset * sigmas[i]))
    shape = numpyro.sample("shape", DISTRIBUTIONS[shape_dist](**shape_params))

    # Expected value
    mu = avg_salary[ind]
    for i, feature in enumerate(features_names):
        mu += priors[i][ind] * X[:,i]

    mu = jnp.exp(mu)
    rate = shape / mu

    # Likelihood
    with numpyro.plate("data", X.shape[0]):
        numpyro.sample("salary_hat", DISTRIBUTIONS[target_dist](concentration=shape, rate=rate), obs=y)

## Define functions

In [6]:
def filter_data(year, data, columns=None):
    # Prepare data for running the model
    if columns is None:
        columns = ["exp","sex","elementary_edu", "highschool_edu", "postsec_edu",
                "undergrad_edu", "graduate_edu", "age", "tenure", "union",
                "part_time", "public_sector", "self_emp"]
    dataset = data.query(f'year == {year}').copy()
    X = dataset[columns].values
    y = dataset["salary"].values
    ind = pd.factorize(dataset["industry"])[0]
    return X, y, ind

In [7]:
def create_model(model_type):
    if model_type == "pooled":
        return pooled
    elif model_type == "no_pooled":
        return no_pooled
    elif model_type == "hierarchical":
        return hierarchical
    else:
        raise ValueError("Invalid model type")

In [8]:
def set_coords(mcmc, dimension, categories, data):
    model_coords = {"coords": {dimension: categories, "obs": np.arange(0,data.shape[0])}}
    model_coords["dims"] = {}
    for latent_var in mcmc._states['z'].keys():
        if any(latent_var.startswith(field) for field in ["avg_","beta_"]):
            model_coords["dims"][latent_var] = [dimension]
    return model_coords

In [9]:
def export_model_outputs(mcmc, model, path, *model_params, **model_coords):
    # Export mcmc
    with open(f"{path}/model.pickle", "wb") as file:
        pickle.dump(mcmc, file)
    # Create posterior predictive samples
    predictive = Predictive(model, mcmc.get_samples())
    posterior_samples = predictive(rng_key, *model_params)
    # Add posterior predictive samples to trace
    if model_coords is None:
        trace = az.from_numpyro(mcmc, posterior_predictive=posterior_samples)
    else:
        trace = az.from_numpyro(mcmc, posterior_predictive=posterior_samples, coords=model_coords["coords"], dims=model_coords["dims"])
    # Export trace
    trace.to_netcdf(f"{path}/trace.nc")
    # Export summary
    summary = az.summary(trace, round_to=5)
    summary.to_csv(f"{path}/summary.csv")
    # Return max Rhat
    return summary["r_hat"].max()   

## Import Data

In [10]:
# Load data and workflow
data = pd.read_csv('../datasets/model_dataset.csv')

In [11]:
# Convert industries and occupations to categorical and create codes columns
ind_cat = [
    'agriculture',
    'forestry/oil/mining',
    'utilities',
    'construction',
    'manufacturing',
    'trade',
    'transportation',
    'info/culture',
    'finance/real estate',
    'scientific/technical',
    'business support',
    'education',
    'health/social',
    'accommodation/food',
    'other services',
    'public admin']
data["industry"] = pd.Categorical(data["industry"], categories=ind_cat)
data["ind_codes"] = data["industry"].cat.codes

occ_cat = ['senior management',
    'middle management',
    'business/finance professional',
    'secretarial/administrative',
    'natural/sciences professional',
    'technical specialist',
    'health professional',
    'health assistant',
    'teachers/professors',
    'government/religion services',
    'protective services',
    'childcare/home support',
    'art/culture occupations',
    'clerical/supervisor',
    'chefs/food services',
    'sales/service',
    'clerks/cashiers',
    'construction trades',
    'transport/equipment operators',
    'trade helper/labourer',
    'trades contractors/supervisors',
    'other trades',
    'operators/assemblers',
    'manufacturing labourer']
data["occup"] = pd.Categorical(data["occup"], categories=occ_cat)
data["occ_codes"] = data["occup"].cat.codes

In [12]:
# Get unique years and sort them
years = data["year"].unique()
years.sort()

# Define features
feature_names = ["exp","sex","elementary_edu", "highschool_edu", "postsec_edu",
"undergrad_edu", "graduate_edu", "age", "tenure", "union",
"part_time", "public_sector", "self_emp"]

In [13]:
# Standardize data
data["exp"] = (data["exp"] - data["exp"].mean()) / data["exp"].std()
data["age"] = (data["age"] - data["age"].mean()) / data["age"].std()
data["tenure"] = (data["tenure"] - data["tenure"].mean()) / data["tenure"].std()

## Run Models

### Pooled

In [35]:
# Run settings
tune = 1000
draws = 1000
accept_prob = 0.95
chains = 4
model_name = "pooled"
model_type = "pooled" # NOTE: pooled, no_pooled, hierarchical

In [ ]:
years = list(range(1996, 2012))
model = create_model(model_type)
for year in years:
    print(f">>>>>>>>>>>>>>>>> year {year} <<<<<<<<<<<<<<<<<<<")
    # Create output folder
    OUTPUT_PATH = f"../outputs/{model_name}/{year}"
    if not os.path.exists(f"{OUTPUT_PATH}"):
                os.makedirs(f"{OUTPUT_PATH}")
    # Filter data
    X, y, ind = filter_data(year, data)
    # Run model
    kernel = NUTS(model, target_accept_prob=accept_prob, init_strategy=init_to_median(num_samples=200))
    if year == 1996:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, feature_names)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        model_params = [X, None, ind, feature_names]
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params)
    else:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, feature_names, samples)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params)
    print(f"Max Rhat: {max_rhat}")

### Pooled | Regularized 

In [40]:
# Run settings
tune = 1000
draws = 1000
accept_prob = 0.95
chains = 4
model_name = "pooled-reg"
model_type = "pooled" # NOTE: pooled, no_pooled, hierarchical
init_params_kwargs = {
    "prior_dist": "laplace",
    "prior_params": {"loc": 0, "scale": 0.01},
    "shape_dist": "uniform",
    "shape_params": {"low": 1, "high": 100},
    "target_dist": "gamma"
}

In [ ]:
years = list(range(1996, 2012))
model = create_model(model_type)
for year in years:
    print(f">>>>>>>>>>>>>>>>> year {year} <<<<<<<<<<<<<<<<<<<")
    # Create output folder
    OUTPUT_PATH = f"../outputs/{model_name}/{year}"
    if not os.path.exists(f"{OUTPUT_PATH}"):
                os.makedirs(f"{OUTPUT_PATH}")
    # Filter data
    X, y, ind = filter_data(year, data)
    # Run model
    rng_key = random.PRNGKey(0)
    rng_key, rng_key_ = random.split(rng_key)
    kernel = NUTS(model, target_accept_prob=accept_prob, init_strategy=init_to_median(num_samples=200))
    if year == 1996:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, feature_names, **init_params_kwargs)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        model_params = [X, None, ind, feature_names]
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params)
    else:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, feature_names, samples, **init_params_kwargs)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params)
    print(f"Max Rhat: {max_rhat}")

### No-pooled

In [14]:
# Run settings
tune = 1000
draws = 1000
accept_prob = 0.95
chains = 4
model_name = "no-pooled"
model_type = "no_pooled" # NOTE: pooled, no_pooled, hierarchical

In [ ]:
years = list(range(1996, 2012))
model = create_model(model_type)
for year in years:
    print(f">>>>>>>>>>>>>>>>> year {year} <<<<<<<<<<<<<<<<<<<")
    # Create output folder
    OUTPUT_PATH = f"../outputs/{model_name}/{year}"
    if not os.path.exists(f"{OUTPUT_PATH}"):
                os.makedirs(f"{OUTPUT_PATH}")
    # Filter data
    X, y, ind = filter_data(year, data)
    # Run model
    rng_key = random.PRNGKey(0)
    rng_key, rng_key_ = random.split(rng_key)
    kernel = NUTS(model, target_accept_prob=accept_prob, init_strategy=init_to_median(num_samples=200))
    if year == 1996:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, feature_names)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        model_params = [X, None, ind, feature_names]
        model_coords = set_coords(mcmc, "industry", ind_cat, X)
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    else:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, feature_names, samples)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    print(f"Max Rhat: {max_rhat}")

### No-pooled | Regularized

In [24]:
# Run settings
tune = 1000
draws = 1000
accept_prob = 0.95
chains = 4
model_name = "no-pooled-reg"
model_type = "no_pooled" # NOTE: pooled, no_pooled, hierarchical
init_params_kwargs = {
    "prior_dist": "laplace",
    "prior_params": {"loc": 0, "scale": 0.01},
    "shape_dist": "uniform",
    "shape_params": {"low": 1, "high": 100},
    "target_dist": "gamma"
}

In [ ]:
years = list(range(1996, 2012))
model = create_model(model_type)
for year in years:
    print(f">>>>>>>>>>>>>>>>> year {year} <<<<<<<<<<<<<<<<<<<")
    # Create output folder
    OUTPUT_PATH = f"../outputs/{model_name}/{year}"
    if not os.path.exists(f"{OUTPUT_PATH}"):
                os.makedirs(f"{OUTPUT_PATH}")
    # Filter data
    X, y, ind = filter_data(year, data)
    # Run model
    rng_key = random.PRNGKey(0)
    rng_key, rng_key_ = random.split(rng_key)
    kernel = NUTS(model, target_accept_prob=accept_prob, init_strategy=init_to_median(num_samples=200))
    if year == 1996:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, feature_names, **init_params_kwargs)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        model_params = [X, None, ind, feature_names]
        model_coords = set_coords(mcmc, "industry", ind_cat, X)
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    else:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, feature_names, samples, **init_params_kwargs)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    print(f"Max Rhat: {max_rhat}")

### Hierarchical

In [32]:
# Run settings
tune = 1000
draws = 1000
accept_prob = 0.95
chains = 4
model_name = "hierarchical"
model_type = "hierarchical" # NOTE: pooled, no_pooled, hierarchical

In [ ]:
years = list(range(1996, 2012))
model = create_model(model_type)
for year in years:
    print(f">>>>>>>>>>>>>>>>> year {year} <<<<<<<<<<<<<<<<<<<")
    # Create output folder
    OUTPUT_PATH = f"../outputs/{model_name}/{year}"
    if not os.path.exists(f"{OUTPUT_PATH}"):
                os.makedirs(f"{OUTPUT_PATH}")
    # Filter data
    X, y, ind = filter_data(year, data)
    # Run model
    rng_key = random.PRNGKey(0)
    rng_key, rng_key_ = random.split(rng_key)
    kernel = NUTS(model, target_accept_prob=accept_prob, init_strategy=init_to_median(num_samples=200))
    if year == 1996:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, feature_names)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        model_params = [X, None, ind, feature_names]
        model_coords = set_coords(mcmc, "industry", ind_cat, X)
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    else:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, feature_names, samples)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    print(f"Max Rhat: {max_rhat}")

### Hierarchical | Regularized

In [36]:
# Run settings
tune = 1000
draws = 1000
accept_prob = 0.95
chains = 4
model_name = "hierarchical-reg"
model_type = "hierarchical" # NOTE: pooled, no_pooled, hierarchical
init_params_kwargs = {
    "mu_dist": "laplace",
    "mu_params": {"loc": 0, "scale": 0.001},
    "sigma_dist": "half_normal",
    "sigma_params": {"scale": 0.001},
    "shape_dist": "uniform",
    "shape_params": {"low": 1, "high": 100},
    "target_dist": "gamma"
}

In [ ]:
years = list(range(1996, 2012))
model = create_model(model_type)
for year in years:
    print(f">>>>>>>>>>>>>>>>> year {year} <<<<<<<<<<<<<<<<<<<")
    # Create output folder
    OUTPUT_PATH = f"../outputs/{model_name}/{year}"
    if not os.path.exists(f"{OUTPUT_PATH}"):
                os.makedirs(f"{OUTPUT_PATH}")
    # Filter data
    X, y, ind = filter_data(year, data)
    # Run model
    rng_key = random.PRNGKey(0)
    rng_key, rng_key_ = random.split(rng_key)
    kernel = NUTS(model, target_accept_prob=accept_prob, init_strategy=init_to_median(num_samples=200))
    if year == 1996:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, feature_names, **init_params_kwargs)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        model_params = [X, None, ind, feature_names]
        model_coords = set_coords(mcmc, "industry", ind_cat, X)
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    else:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, feature_names, samples, **init_params_kwargs)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    print(f"Max Rhat: {max_rhat}")

## Run models (variable selection)

### Variable Selection | VS1 - NO self_emp

In [11]:
# Run settings
tune = 1000
draws = 1000
accept_prob = 0.95
chains = 4
model_name = "VS1-no_self_emp"
model_type = "hierarchical" # NOTE: pooled, no_pooled, hierarchical

In [ ]:
years = list(range(1996, 2012))
model = create_model(model_type)
for year in years:
    print(f">>>>>>>>>>>>>>>>> year {year} <<<<<<<<<<<<<<<<<<<")
    # Create output folder
    OUTPUT_PATH = f"../outputs/{model_name}/{year}"
    if not os.path.exists(f"{OUTPUT_PATH}"):
                os.makedirs(f"{OUTPUT_PATH}")
    # Filter data
    columns = ["exp","sex","elementary_edu", "highschool_edu", "postsec_edu",
                "undergrad_edu", "graduate_edu", "age", "tenure", "union",
                "part_time", "public_sector"]
    X, y, ind = filter_data(year, data, columns=columns)
    # Run model
    rng_key = random.PRNGKey(0)
    rng_key, rng_key_ = random.split(rng_key)
    kernel = NUTS(model, target_accept_prob=accept_prob, init_strategy=init_to_median(num_samples=200))
    if year == 1996:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, columns)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        model_params = [X, None, ind, columns]
        model_coords = set_coords(mcmc, "industry", ind_cat, X)
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    else:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, columns, samples)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    print(f"Max Rhat: {max_rhat}")

### Variable Selection | VS2 - NO public sector

In [13]:
# Run settings
tune = 1000
draws = 1000
accept_prob = 0.95
chains = 4
model_name = "VS2-no_public_sector"
model_type = "hierarchical" # NOTE: pooled, no_pooled, hierarchical

In [ ]:
compilate_samples = []
years = list(range(1996, 2012))
model = create_model(model_type)
for year in years:
    print(f">>>>>>>>>>>>>>>>> year {year} <<<<<<<<<<<<<<<<<<<")
    # Create output folder
    OUTPUT_PATH = f"../outputs/{model_name}/{year}"
    if not os.path.exists(f"{OUTPUT_PATH}"):
                os.makedirs(f"{OUTPUT_PATH}")
    # Filter data
    columns = ["exp","sex","elementary_edu", "highschool_edu", "postsec_edu",
                "undergrad_edu", "graduate_edu", "age", "tenure", "union",
                "part_time"]
    X, y, ind = filter_data(year, data, columns=columns)
    # Run model
    rng_key = random.PRNGKey(0)
    rng_key, rng_key_ = random.split(rng_key)
    kernel = NUTS(model, target_accept_prob=accept_prob, init_strategy=init_to_median(num_samples=200))
    if year == 1996:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, columns)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        model_params = [X, None, ind, columns]
        model_coords = set_coords(mcmc, "industry", ind_cat, X)
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    else:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, columns, samples)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    print(f"Max Rhat: {max_rhat}")

### Variable Selection | VS3 - NO part_time

In [15]:
# Run settings
tune = 1000
draws = 1000
accept_prob = 0.95
chains = 4
model_name = "VS3-no_part_time"
model_type = "hierarchical" # NOTE: pooled, no_pooled, hierarchical

In [ ]:
compilate_samples = []
years = list(range(1996, 2012))
model = create_model(model_type)
for year in years:
    print(f">>>>>>>>>>>>>>>>> year {year} <<<<<<<<<<<<<<<<<<<")
    # Create output folder
    OUTPUT_PATH = f"../outputs/{model_name}/{year}"
    if not os.path.exists(f"{OUTPUT_PATH}"):
                os.makedirs(f"{OUTPUT_PATH}")
    # Filter data
    columns = ["exp","sex","elementary_edu", "highschool_edu", "postsec_edu",
                "undergrad_edu", "graduate_edu", "age", "tenure", "union"]
    X, y, ind = filter_data(year, data, columns=columns)
    # Run model
    rng_key = random.PRNGKey(0)
    rng_key, rng_key_ = random.split(rng_key)
    kernel = NUTS(model, target_accept_prob=accept_prob, init_strategy=init_to_median(num_samples=200))
    if year == 1996:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, columns)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        model_params = [X, None, ind, columns]
        model_coords = set_coords(mcmc, "industry", ind_cat, X)
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    else:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, columns, samples)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    print(f"Max Rhat: {max_rhat}")

### Variable Selection | VS4 - NO union

In [17]:
# Run settings
tune = 1000
draws = 1000
accept_prob = 0.95
chains = 4
model_name = "VS4-no_union"
model_type = "hierarchical" # NOTE: pooled, no_pooled, hierarchical

In [ ]:
compilate_samples = []
years = list(range(1996, 2012))
model = create_model(model_type)
for year in years:
    print(f">>>>>>>>>>>>>>>>> year {year} <<<<<<<<<<<<<<<<<<<")
    # Create output folder
    OUTPUT_PATH = f"../outputs/{model_name}/{year}"
    if not os.path.exists(f"{OUTPUT_PATH}"):
                os.makedirs(f"{OUTPUT_PATH}")
    # Filter data
    columns = ["exp","sex","elementary_edu", "highschool_edu", "postsec_edu",
                "undergrad_edu", "graduate_edu", "age", "tenure"]
    X, y, ind = filter_data(year, data, columns=columns)
    # Run model
    rng_key = random.PRNGKey(0)
    rng_key, rng_key_ = random.split(rng_key)
    kernel = NUTS(model, target_accept_prob=accept_prob, init_strategy=init_to_median(num_samples=200))
    if year == 1996:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, columns)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        model_params = [X, None, ind, columns]
        model_coords = set_coords(mcmc, "industry", ind_cat, X)
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    else:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, columns, samples)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    print(f"Max Rhat: {max_rhat}")

### Variable Selection | VS5 - NO tenure

In [19]:
# Run settings
tune = 1000
draws = 1000
accept_prob = 0.95
chains = 4
model_name = "VS5-no_tenure"
model_type = "hierarchical" # NOTE: pooled, no_pooled, hierarchical

In [ ]:
compilate_samples = []
years = list(range(1996, 2012))
model = create_model(model_type)
for year in years:
    print(f">>>>>>>>>>>>>>>>> year {year} <<<<<<<<<<<<<<<<<<<")
    # Create output folder
    OUTPUT_PATH = f"../outputs/{model_name}/{year}"
    if not os.path.exists(f"{OUTPUT_PATH}"):
                os.makedirs(f"{OUTPUT_PATH}")
    # Filter data
    columns = ["exp","sex","elementary_edu", "highschool_edu", "postsec_edu",
                "undergrad_edu", "graduate_edu", "age"]
    X, y, ind = filter_data(year, data, columns=columns)
    # Run model
    rng_key = random.PRNGKey(0)
    rng_key, rng_key_ = random.split(rng_key)
    kernel = NUTS(model, target_accept_prob=accept_prob, init_strategy=init_to_median(num_samples=200))
    if year == 1996:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, columns)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        model_params = [X, None, ind, columns]
        model_coords = set_coords(mcmc, "industry", ind_cat, X)
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    else:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, columns, samples)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    print(f"Max Rhat: {max_rhat}")

### Variable Selection | VS6 - NO Age

In [21]:
# Run settings
tune = 1000
draws = 1000
accept_prob = 0.95
chains = 4
model_name = "VS6-no_age"
model_type = "hierarchical" # NOTE: pooled, no_pooled, hierarchical

In [ ]:
compilate_samples = []
years = list(range(1996, 2012))
model = create_model(model_type)
for year in years:
    print(f">>>>>>>>>>>>>>>>> year {year} <<<<<<<<<<<<<<<<<<<")
    # Create output folder
    OUTPUT_PATH = f"../outputs/{model_name}/{year}"
    if not os.path.exists(f"{OUTPUT_PATH}"):
                os.makedirs(f"{OUTPUT_PATH}")
    # Filter data
    columns = ["exp","sex","elementary_edu", "highschool_edu", "postsec_edu",
                "undergrad_edu", "graduate_edu"]
    X, y, ind = filter_data(year, data, columns=columns)
    # Run model
    rng_key = random.PRNGKey(0)
    rng_key, rng_key_ = random.split(rng_key)
    kernel = NUTS(model, target_accept_prob=accept_prob, init_strategy=init_to_median(num_samples=200))
    if year == 1996:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, columns)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        model_params = [X, None, ind, columns]
        model_coords = set_coords(mcmc, "industry", ind_cat, X)
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    else:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, columns, samples)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    print(f"Max Rhat: {max_rhat}")

### Variable Selection | VS7 - No Education Level

In [23]:
# Run settings
tune = 1000
draws = 1000
accept_prob = 0.95
chains = 4
model_name = "VS7-no_edu"
model_type = "hierarchical" # NOTE: pooled, no_pooled, hierarchical

In [ ]:
compilate_samples = []
years = list(range(1996, 2012))
model = create_model(model_type)
for year in years:
    print(f">>>>>>>>>>>>>>>>> year {year} <<<<<<<<<<<<<<<<<<<")
    # Create output folder
    OUTPUT_PATH = f"../outputs/{model_name}/{year}"
    if not os.path.exists(f"{OUTPUT_PATH}"):
                os.makedirs(f"{OUTPUT_PATH}")
    # Filter data
    columns = ["exp","sex"]
    X, y, ind = filter_data(year, data, columns=columns)
    # Run model
    rng_key = random.PRNGKey(0)
    rng_key, rng_key_ = random.split(rng_key)
    kernel = NUTS(model, target_accept_prob=accept_prob, init_strategy=init_to_median(num_samples=200))
    if year == 1996:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, columns)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        model_params = [X, None, ind, columns]
        model_coords = set_coords(mcmc, "industry", ind_cat, X)
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    else:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, columns, samples)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    print(f"Max Rhat: {max_rhat}")

### Variable Selection | VS8 - No Sex

In [25]:
# Run settings
tune = 1000
draws = 1000
accept_prob = 0.95
chains = 4
model_name = "VS8-no_sex"
model_type = "hierarchical" # NOTE: pooled, no_pooled, hierarchical

In [ ]:
compilate_samples = []
years = list(range(1996, 2012))
model = create_model(model_type)
for year in years:
    print(f">>>>>>>>>>>>>>>>> year {year} <<<<<<<<<<<<<<<<<<<")
    # Create output folder
    OUTPUT_PATH = f"../outputs/{model_name}/{year}"
    if not os.path.exists(f"{OUTPUT_PATH}"):
                os.makedirs(f"{OUTPUT_PATH}")
    # Filter data
    columns = ["exp"]
    X, y, ind = filter_data(year, data, columns=columns)
    # Run model
    rng_key = random.PRNGKey(0)
    rng_key, rng_key_ = random.split(rng_key)
    kernel = NUTS(model, target_accept_prob=accept_prob, init_strategy=init_to_median(num_samples=200))
    if year == 1996:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, columns)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        model_params = [X, None, ind, columns]
        model_coords = set_coords(mcmc, "industry", ind_cat, X)
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    else:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, columns, samples)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    print(f"Max Rhat: {max_rhat}")

### Variable Selection | VS9 - No exp

In [12]:
# Run settings
tune = 1000
draws = 1000
accept_prob = 0.95
chains = 4
model_name = "VS9-no_exp"
model_type = "hierarchical" # NOTE: pooled, no_pooled, hierarchical

In [ ]:
compilate_samples = []
years = list(range(1996, 2012))
model = create_model(model_type)
for year in years:
    print(f">>>>>>>>>>>>>>>>> year {year} <<<<<<<<<<<<<<<<<<<")
    # Create output folder
    OUTPUT_PATH = f"../outputs/{model_name}/{year}"
    if not os.path.exists(f"{OUTPUT_PATH}"):
                os.makedirs(f"{OUTPUT_PATH}")
    # Filter data
    columns = []
    X, y, ind = filter_data(year, data, columns=columns)
    # Run model
    rng_key = random.PRNGKey(0)
    rng_key, rng_key_ = random.split(rng_key)
    kernel = NUTS(model, target_accept_prob=accept_prob, init_strategy=init_to_median(num_samples=200))
    if year == 1996:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, columns)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        model_params = [X, None, ind, columns]
        model_coords = set_coords(mcmc, "industry", ind_cat, X)
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    else:
        mcmc = MCMC(kernel, num_warmup=tune, num_samples=draws, num_chains=chains, chain_method="vectorized")
        mcmc.run(rng_key, X, y, ind, columns, samples)
        samples = mcmc.get_samples()
        # Save model outputs and calculate max Rhat
        max_rhat = export_model_outputs(mcmc, model, OUTPUT_PATH, *model_params, **model_coords)
    print(f"Max Rhat: {max_rhat}")